The purpuse of this notebook is to provide an implementation of a polynomial-time key-recovery attack against the Sidon cryptosystem [1]. Also, several experiments are implemented to verify the polynomial-complexity of the attack. 

[1] Netanel Raviv, Ben Langton, and Itzhak Tamo. Multivariate public key cryptosys-tem from sidon spaces. In Juan A. Garay, editor,Public-Key Cryptography – PKC2021, pages 242–265, Cham, 2021. Springer International Publishing

## Original Implementation of the Sidon Cryptosystem

In [ ]:
def ConstructSidon(q, k, r): 
    ##Constructs a Sidon Space in Gq(rk, k) for r > 2 
    
    ##Define the field F_p^k and the polynomial ring over that field
    assert(r!=2)
    F.<z> = GF(q^k)
    R.<a> = F[]
    
    ##Construct another extension field of degree r over this ring using an irreducible polynomial of degree r over F_q^k
    irred_poly1 = R.irreducible_element(r)
    irred_poly2 = R.irreducible_element(r)
    F_r.<x> = F.extension(irred_poly1)[]
    F_ = F.extension(irred_poly1)
    ##find a root of this in F_q^rk
    roots = irred_poly2(x).roots()

    y = roots[0][0]
    
    ##returns a tuple containing y, the subfield F_q^k and the field F_q^rk
    ##The Sidon space is defined as u + u^p*y for u in F 
    return y, F, F_

In [ ]:
def ConstructSidon2k(q, k): 
    ##Constructs a Sidon Space in Gq(2k, k), q cannot equal 2 
    assert(q!=2)
    
    ##Define the field F_q^k and the polynomial ring
    
    F.<z> = GF(q^k) 
    R.<a> = F[] 
    
    p = F.characteristic()
    ##Find and irreducible polynomial of degree 2 with the constant term not in W_q-1 
    irred_poly = R.irreducible_element(2)
    iterations = 0
   
    while irred_poly.list()[0]^((q^k - 1)/(q-1)) == 1 and iterations < 100: 
        irred_poly = R.irreducible_element(2)
        iterations += 1
    
    assert(iterations != 100)
    
  
    ##Construct the second extension field as a polynomial ring over the first modded out by this irreducible element
    X  = F.extension(irred_poly)
    F_2k.<x> = F.extension(irred_poly)[]
    roots = irred_poly(x).roots() 
    
    ##Return a root of the polynomial as well as info about the two fields 
    y = roots[0][0]
    
    return y, F, X, irred_poly.list()[1], irred_poly.list()[0]
    
    
    

In [ ]:
def factor(product, y, q, F, F_r): 
    ##Factoring algorithm for Sidon spaces with r > 2 
    r = len(list(F_r.modulus())) - 1 
    p = F.characteristic()
    assert(r > 2)
    
    ##construct a basis from y 
    basis = [vector(y^n) for n in range(r)]
    mat = matrix(basis)
    
    ##calculate the change of basis matrix to go from the standard basis to our new basis, and compute 
    ##the representation of the product in that basis 
    cob_mat = mat.inverse() 
    product_representation = vector(product)*cob_mat
    
    ##Find the roots of the polynomial we derive from product_represntation, and then compute the original u and v
    assert(i == 0 for i in product_representation[3:])
    product_representation = product_representation[0:3]
    F_.<x> = F[]
    poly = F_(list(product_representation))
    roots = poly.roots()
    
    ##returns u and v up to multiplication from F_p, not the original things multiplied 
    if len(roots) == 1: 
        ans1 = (-1*1/roots[0][0]).nth_root(q - 1)
        ans2 = (-1*1/roots[0][0]).nth_root(q - 1)
        
    else: 
        ans1 = (-1*1/roots[0][0]).nth_root(F.characteristic() - 1)
        ans2 = (-1*1/roots[1][0]).nth_root(F.characteristic() - 1)
    
    if product/((ans1 + ans1^q*y)*(ans2 + ans2^q*y)) != 1: 
        ans1 = ans1*product/((ans1 + ans1^q*y)*(ans2 + ans2^q*y))
    return ans1, ans2

In [ ]:
def factor2(product, y,q,F, F_r, b, c): 
    ##factoring algorithm for Sidon space with r = 2 
    p = F.characteristic()
    k = len(F.modulus().list()) - 1
    
    ##construct a basis for F_r that we will use to extract usefull info
    basis = [vector(y^n) for n in (0, 1)]
    mat = matrix(basis)
    cob_mat = mat.inverse() 
    product_representation = vector(product)*cob_mat
    
    
    ##Figure out the linear transformation x - cx^q 
    basis_transf = []
    identity = matrix.identity(k)
    
    for i in range(k): 
        transformed = vector(F(list(identity[i])) - c*F(list(identity[i]))^q)
        basis_transf += [transformed]
        
    transformation = matrix(basis_transf).inverse() 
    
    ##invert this transformation and use it to calculate uv
    #print(vector(product_representation[0]))
    uv = vector(product_representation[0])*transformation
    uv = F(uv)
 
    ##Calculate the last two terms and create a quadratic polynomial
    F_.<s> = F[] 
    second_term = product_representation[1] + b*(uv)^q
    third_term = uv^q
    poly = F_([vector(uv), vector(second_term), vector(third_term)])
    
    ##get the roots of the polynomial and extract u and v
    roots = poly.roots()
    
    ##returns u and v up to multiplication from F_q, not the original things multiplied 
    if len(roots) == 1: 
        ans1 = (-1*1/roots[0][0]).nth_root(q - 1)
        ans2 = (-1*1/roots[0][0]).nth_root(q - 1)
        
    else: 
        ans1 = (-1*1/roots[0][0]).nth_root(q - 1)
        ans2 = (-1*1/roots[1][0]).nth_root(q - 1)
    
    if product/((ans1 + ans1^q*y)*(ans2 + ans2^q*y)) != 1: 
        ans1 = ans1*product/((ans1 + ans1^q*y)*(ans2 + ans2^q*y))
    return ans1, ans2
    

In [ ]:
def publicKey(y,q, F, F_r): 
    ##Generates the public key (and associated private key) using info from the given sidon space 
    p = F.characteristic()
    k = len(F.modulus().list()) - 1
    #print(k)
    rk = (len(F_r.modulus().list()) - 1)*k
    basefield = GF(q)
    iterations = 0
    iterations2 = 0
    
    ##Construct bases for the sidon space as well as F_r
    sidonbasis = Matrix(basefield, k, lambda i,j: basefield.random_element())
    while sidonbasis.is_invertible() == False and iterations < 100:
        sidonbasis = Matrix(basefield, k, lambda i,j: basefield.random_element())
        iterations += 1
    F_r_basis = Matrix(basefield, rk, lambda i,j: basefield.random_element())
    while F_r_basis.is_invertible() == False and iterations2 < 100: 
         F_r_basis = Matrix(basefield, rk, lambda i,j: basefield.random_element())
         iterations2 += 1
    
    assert(iterations<100 and iterations2<100)
    
    ##Get the multiplication table from the basis of the sidon space
    #print(sidonbasis[0])
    v = [F(list(sidonbasis[i])) for i in range(k)]
    origbasis = sidonbasis
    sidonbasis = [j + j^q*y for j in v]
    mult_table = vector(sidonbasis).column()*vector(sidonbasis).row()
    cob_matrix = F_r_basis.inverse()
    #print(cob_matrix)
    vec_list = [[0 for i in range(k)] for j in range(k)]
    
    ##Generate the public key M(V,B)
    for i in range(k): 
        for j in range(k):
            element = mult_table[i][j]
            long_representation = convertToLong(element)
            vec_list[i][j] = list(long_representation*cob_matrix) 
    matrixlist = [0 for i in range(rk)]
    
    ##Return the public key 
    for i in range(rk): 
        matrixlist[i] = Matrix(basefield, k, lambda l,j: vec_list[l][j][i])
    return matrixlist, sidonbasis, mult_table, F_r_basis, origbasis

def convertToLong(element):
    ##Converts an element from a vector over F to a vector over F_q
    long_representation = []
    #print(list(element))
    for l in range(len(list(element))): 
        long_representation += list(vector(list(element)[l]))
    long_representation = vector(long_representation)
    long_representation
    return long_representation

def convertFromLong(element, F, F_r):
    ##Converts an element from F_r represented as a vector over F_q to a vector over F
    
    k = len(F.modulus().list()) - 1
    r = len(F_r.modulus().list()) - 1
    final_list = []
    
    for i in range(r): 
        final_list += [F(list(element)[i*k:(i+1)*k])]
    return(F_r(final_list))
    
    

In [ ]:
def encrypt(a,b, matrixlist): 
    ##Uses the public key to send the encrypted message
    anslist = []
    #print(a,b,matrixlist)
    for i in matrixlist: 
        anslist += a.row()*i*b.column()
    return anslist 


def decrypt(anslist, y, q, F, F_r, F_r_basis, sidonbasis, origbasis, b = None, c = None): 
    ##Decrpyts the message anslist
    ##Calculate the original product in the sidon space
    product = (anslist[0]*convertFromLong(F_r_basis[0],F,F_r))
    
    for i in range(len(anslist) - 1): 
        product += anslist[i+1]*convertFromLong(F_r_basis[i+1], F, F_r)
    r = len(F_r.modulus().list()) - 1 
    
    ##Factor the product
    if r == 2: 
        u,v = factor2(product[0], y,q, F, F_r, b, c)
    else: 
        u,v = factor(product[0], y,q, F, F_r)
    #print(product)
    #print((u + u^q*y)*(v + v^q*y))
    
    ##Represent the product over the sidon space basis
    cob_matrix = origbasis.inverse() 
  
    u = vector(F(str(u)))*origbasis.inverse()
    v = vector(F(str(v)))*origbasis.inverse() 
    
   
    return u.column()*v.row()


        

In [ ]:
##demonstration
q = 5
k = 8
#r = 4
y, F, F_r, b, c = ConstructSidon2k(q, k)
matrixlist, sidonbasis, mult_table, F_r_basis , origbasis = publicKey(y,q,F,F_r)

In [ ]:
plaintext1 = vector(GF(q), [GF(q).random_element() for _ in range(k)])
plaintext2 = vector(GF(q), [GF(q).random_element() for _ in range(k)])
ciphertext = encrypt(plaintext1, plaintext2, matrixlist)
decrypt(ciphertext, y, q, F, F_r, F_r_basis, sidonbasis, origbasis, b, c)

# From here our functions

In [ ]:
def frob(u):
    return [u[i]**q for i in range(k)]

# MinRank Part

In [ ]:
def find_minrank_solutions(M_u, Fqn):
    """
    Return the MinRank coefficients over Fqn for a given rank one matrix M_u in the span of the public matrices.
    """
    k = matrixlist[0].dimensions()[0]
    MR = PolynomialRing(Fqn, 2 * k, 'm')
    linear_eq_to_solve_minrank = list(vector(M_u) - vector(sum([matrixlist[i] * MR.gen(i) for i in range(2 * k)])))
    minrank_solutions = ideal(linear_eq_to_solve_minrank).variety()
    minrank_solutions_list = []
    for sol in minrank_solutions:
        sol_list = 2*k*[0]
        for x, value in sol.items():
            sol_list[int(str(x)[1:])] = value
        minrank_solutions_list.append(vector(sol_list))
    return minrank_solutions_list

## Modeling 1

In [ ]:
def modeling_1(matrixlist):
    """
    Return the equations of Modeling 1. It takes as input the set of public symmetric matrices matrixlist.
    """
    base_field = matrixlist[0].base_ring()
    q = base_field.characteristic()
    k = matrixlist[0].dimensions()[0]
    M = Matrix(Fqn, [vector(matrixlist[i]) for i in range(2*k)])
    K = M.transpose().kernel()
    R = PolynomialRing(Fqn, k, 'x')
    vector_x = vector(R.gens())
    monom1 = vector(vector_x.column() * vector_x.row())
    Ker = K.basis_matrix()
    mat_equa1 = Ker * monom1.column()
    equations_modeling1 = list(set([mat_equa1[i][0] for i in range(k**2 - 2*k)])) 
    return equations_modeling1


def solve_modeling_1(matrixlist, Fqn, variables_to_fix={0:1, 1:0}):
    """
    Returns all solutions of Modeling 1
    """
    k = matrixlist[0].dimensions()[0]
    R = PolynomialRing(Fqn, k, 'x', order = "lex")
    equations_modeling1 = modeling_1(matrixlist)
    fixed_variables = []
    for key, value in variables_to_fix.items():
        fixed_variables.append(R.gen(key) - value)
    equa = equations_modeling1 + fixed_variables
    I = ideal(R, equa)
    G = I.groebner_basis("magma")
    solutions = ideal(G).variety() 
    sol_list = k*[0]
    solutions_list = []
    for sol in solutions:
        sol_list = k*[0]
        for x, value in sol.items():
            sol_list[int(str(x)[1])] = value
        solutions_list.append(vector(sol_list))
    return solutions_list

In [ ]:
##demonstration
q = 5
k = 8
#r = 4
y, F, F_r, d, c = ConstructSidon2k(q, k)
matrixlist, sidonbasis, mult_table, F_r_basis , origbasis = publicKey(y,q,F,F_r)
Fq = GF(q) 
Fqn = Fq.extension(2 * k) 
Fqk = Fqn.subfield(k)
solve_modeling_1(matrixlist, Fqn, variables_to_fix={0:1, 1:0})

# Finding a rank one matrix of the form $(\textbf{u} + \gamma \textbf{u}^{[1]})^{\top}(\textbf{u} + \gamma \textbf{u}^{[1]})$

# Key recovery attack

In [ ]:
def get_equivalent_key(matrixlist, Fqn):
    Fqk = Fqn.subfield(k)  
    #We solve twice modeling 1.
    solutions1 = solve_modeling_1(matrixlist, Fqn, variables_to_fix={0:Fqk.random_element(), 1:Fqk.random_element()})
    solutions2 = solve_modeling_1(matrixlist, Fqn, variables_to_fix={0:Fqk.random_element(), 1:Fqk.random_element()})
    solutions = solutions1 + solutions2
    a = 0
    for i in range(2 * k):
        u0 = solutions[i];
        for j in range(i + 1, 2 * k):
            v0 = solutions[j]
            if matrix([u0, v0 , frob(u0), frob(v0)]).rank() < 4:
                u1 = solutions[i]
                v1 = solutions[j]
                K = matrix([u1, v1 , frob(u1), frob(v1)]).kernel().basis_matrix()
                important_u = K[0][0] * u1 + K[0][1] * v1
                #important_v = vector([i**q for i in u])

                RR.<a> = Fqk[]
                ##Find and irreducible polynomial of degree 2 with the constant term not in W_q-1 
                irred_poly = RR.irreducible_element(2)
                iterations = 0

                while irred_poly.list()[0]^((q^k - 1)/(q-1)) == 1 and iterations < 100: 
                    irred_poly = RR.irreducible_element(2)
                    iterations += 1

                assert(iterations != 100)

                ##Construct the second extension field as a polynomial ring over the first modded out by this irreducible element
                X  = Fqk.extension(irred_poly)
                F_2k.<x> = Fqk.extension(irred_poly)[]
                roots = irred_poly(x).roots() 

                ##Return a root of the polynomial as well as info about the two fields 
                y_recovered = roots[0][0]



                recovered_sidonbasis = [j + j^q*y_recovered for j in important_u]
                a = 1
            if a:
                break

        if a:
            break
    origbasis_recovered = matrix([vector(Fqk(important_u[i])) for i in range(k)])
    recovered_M = vector(recovered_sidonbasis).column() * vector(recovered_sidonbasis).row()
   #M_u = important_u.column() * important_u.row()
    #v = frob(important_u)
    #M_v = vector(v).column() * vector(v).row()
    F_r_basis_recovered = find_minrank_solutions(recovered_M, X)
    b_recovered, c_recovered = irred_poly.list()[1], irred_poly.list()[0]
    F_r_recovered = X
    return y_recovered, Fqk,  F_r_recovered, F_r_basis_recovered, recovered_sidonbasis, origbasis_recovered,b_recovered, c_recovered
    

In [ ]:
def new_decrypt(y_rec, Fqk_rec, F_r_rec, F_r_basis_rec, sidonbasis_rec, origbasis_rec, b_rec, c_rec):
    product = sum([ciphertext[i] * F_r_basis_rec[0][i] for i in range(2 * k)])
    r = len(F_r_rec.modulus().list()) - 1 
    ##Factor the product
    if r == 2: 
        u,v = factor2(product[0], y_rec,q, Fqk_rec, F_r_rec, b_rec, c_rec)
    else: 
        u,v = factor(product[0], y_rec,q, Fqk_rec, F_r_rec)

    ##Represent the product over the sidon space basis
    cob_matrix = origbasis_rec.inverse() 

    u = vector(Fqk_rec(str(u)))*cob_matrix
    v = vector(Fqk_rec(str(v)))*cob_matrix


    return v.column() * u.row()

# Testing

In [ ]:
q = 5
k = 7
#r = 4
y, F, F_r, b, c = ConstructSidon2k(q, k)
matrixlist, sidonbasis, mult_table, F_r_basis , origbasis = publicKey(y,q,F,F_r)
Fq = GF(q) 
Fqn = Fq.extension(2 * k) 

In [ ]:
y_rec,Fqk_rec,F_r_rec,F_r_basis_rec,sidonbasis_rec,origbasis_rec,b_rec,c_rec=get_equivalent_key(matrixlist, Fqn)

In [ ]:
plaintext1 = vector(GF(q), [GF(q).random_element() for _ in range(k)])
plaintext2 = vector(GF(q), [GF(q).random_element() for _ in range(k)])
ciphertext = encrypt(plaintext1, plaintext2, matrixlist)
recovered_matrix = new_decrypt(y_rec,Fqk_rec,F_r_rec,F_r_basis_rec,sidonbasis_rec,origbasis_rec,b_rec,c_rec) 
original_matrix = decrypt(ciphertext, y, q, F, F_r, F_r_basis, sidonbasis, origbasis, b, c)

In [ ]:
print((recovered_matrix == original_matrix) | (recovered_matrix == original_matrix.transpose()))

# On the Complexity of Modeling 1.

### Testing the Solving Degree.

In [ ]:
def gb_modeling_1(matrixlist, Fqk):
    """
    Returns a Groebner basis in the grevlex order of the ideal generated by the Modeling 1 equations.
    """
    equations_modeling1 = modeling_1(matrixlist)
    k = matrixlist[0].dimensions()[0]
    R = PolynomialRing(Fqk, k, 'x')
    fixed_variables = [R.gen(0) - Fqk.random_element(), R.gen(1) - Fqk.random_element()]
    equa = equations_modeling1 + fixed_variables
    G = ideal(equa).groebner_basis("magma", prot = "sage")
    return G

In [ ]:
q = 5
k = 8
#r = 4
y, F, F_r, d, c = ConstructSidon2k(q, k)
matrixlist, sidonbasis, mult_table, F_r_basis , origbasis = publicKey(y,q,F,F_r)
Fq = GF(q) 
Fqn = Fq.extension(2 * k) 
Fqk = Fqn.subfield(k) 
G = gb_modeling_1(matrixlist, Fqk)

## Cheking: Dimension of Modeling 1 equations is $k^2 - 2*k - \binom{k}{2}$

In [ ]:
for k in range(2, 14):
    q = 7
    y, F, F_r, d, c = ConstructSidon2k(q, k)
    matrixlist, sidonbasis, mult_table, F_r_basis , origbasis = publicKey(y,q,F,F_r)
    M = Matrix(F, [vector(matrixlist[i]) for i in range(2*k)])
    K = M.transpose().kernel()
    R = PolynomialRing(F, k, 'x')
    var = R.gens()
    monom = vector(vector(R, var).column() * vector(R,var).row())
    Ker = K.basis_matrix()
    mat_equa = Ker * monom.column()
    equa = list(set([mat_equa[i][0] for i in range(k**2 - 2*k)])) #+ [R.gen(k-1) - F.random_element()] + [R.gen(k-2) - F.random_element()]
    print(len(equa) -1 ==k^2 - 2*k - binomial(k,2))